# Building Bioinformatics Workflows with SnakeMake

## BST 281 Guest Lecture
### April 28, 2025

# Why Learn Workflow Languages?

<ul>
    <li>They allow you to build <b>modular</b>, <b>reproducible</b>, and <b>scalable</b> workflows</li>
    <li>Different processes can be parallelized and resource allocation optimized.</li>
    <li>Popular among bioinformaticians across industries, making it a great transferable skill.</li>
</ul>

<a href="https://snakemake.readthedocs.io/en/stable/" target="_blank"><b>Snakemake</b></a> has Python-based syntax and integrates well with conda environments, making it relatively easy to learn. 

Another popular workflow language is <a href="https://www.nextflow.io/" target="_blank"><b>Nextflow</b></a>.

# Worfklow Components:

There are 2 major components of a snakemake workflow:

1. Rules
2. Snakefile

The rules define the steps of the workflow, and the snakefile defines the aggregate workflow. We'll get into the details below.

# Rules

Each step in a snakemake pipeline is called a <b>rule</b>.

In addition to the exact code to run, each rule defines the input and output files, additional parameters, conda environments (if applicable).

The following is template for a simple rule:

```snakemake
rule myrule:
    input:
        fName1 = "path/to/inputfile",
        fName2 = "path/to/other/inputfile",
    output:
        output1 = "path/to/outputfile",
        output2 = "path/to/another/outputfile",
    shell:
        """
        somecommand {input.fName1} {output.output1}
        somecommand {input.fName1} {output.output2}
        """
```

The commands in the `shell` block are executed as they would be on the command line. You can also run Python itself, with the `run` block:

```snakemake
rule myrule:
    input:
        fName1 = "path/to/inputfile",
        fName2 = "path/to/other/inputfile",
    output:
        output1 = "path/to/outputfile",
        output2 = "path/to/another/outputfile",
    run:
        # this is a python command
        print(f"Filename 1: {input.fName1}, Filename 2: {input.fName2}")
```

You can access all components of the `input`, `output`, and other blocks using the syntax `block_name.attribute`, without the `{}`. However, because I used an f-string above, I included the braces as required by the f-string format.

Importantly, if you define an output file in the `output` block, that file must be generated by the rule, otherwise snakemake will give you an error.

# Snakefile

The rules that will be used in the workflow need to be imported, similar to the `#include` statement in C++ programs.

After that, you define the desired outputs, like so:

```
import pandas as pd

output_dir = config["output_dir"]

# Import rules
include: "rules.smk"

# read in a dataframe of samples to process
df_samples_runs = pd.read_csv(config['isolates_to_run'])

rule all:
    input:
        [f"{output_dir}/{sample}/output_file.txt" for sample in df_samples_runs['Sample'].values]

```

Here, I defined a list of samples to run the workflow on from a dataframe. Because Snakemake is python-based, you can import python packages like `pandas` and read them in as you normally would in python.

Also note that I referenced a config file. The `config` variable is a global variable that is accessible in both the snakefile and the rules file.

# Conda Integration

Since many packages have different dependencies, it is useful to be able to have different virtual environments for different steps. Snakemake allows defining different conda environments like so:

```snakemake
rule myrule:
    input:
        fName1 = "path/to/inputfile",
        fName2 = "path/to/other/inputfile",
    output:
        output1 = "path/to/outputfile",
        output2 = "path/to/another/outputfile",
    shell:
        """
        somecommand {input.fName1} {output.output1}
        somecommand {input.fName1} {output.output2}
    conda:
        "envs/myenv.yaml"
```

# Putting it all together

Once you have all of your rules, you define the workflow by the desired outputs. Snakemake then back fills all the rules that must be run, depending on the inputs and outputs of each rule, to get to the user-specified outputs.

To run a workflow from the command line, the basic syntax is

```bash
snakemake --snakefile snakefile --use-conda --conda-frontend conda --configfile config.yaml --cores 1

```

You MUST specify the number of cores, otherwise snakemake will give you an error.

In the above code, I specified the snakefile and config file to use. You can also change things like the home directory for a workflow (so all relative paths will be relative to this directory) with the `--directory` flag and specify a different directory where conda environments are stored than in this home directory with the `--conda-prefix` flag.

You can also set the maximum available RAM with something like `--resources mem_mb=8000`, meaning that there are 8 GB of RAM available. 

We won't use them, but there are some additional helpful flags:

<ul>
    <li><code>--unlock</code>: Snakemake locks a directory in which a workflow is running, so you can not run multiple workflows from the same directory to prevent potential conflicts in the output files. The directory is unlocked once the process completes. Use this flag to unlock a directory, for example, if a process was interrupted.</li>
    <li><code>--rerun-incomplete</code>: If a snakemake process is interrupted and some steps (jobs) only partially finished, rerun those</li>
    <li><code>--keep-going</code>: If one job fails, but there are other jobs that do not rely on its outputs, run them. This is useful if you have many samples, and one fails, but the workflow can continue processing the others.</li>
    <li><code>--dry-run</code>: Perform a dry run of the worfklow to ensure that the graph of jobs is valid and that all required inputs are present.</li>
</ul>

# Directed Acylic Graph (DAG) of Jobs

To see all the rules that will be run in their order to get to the outputs, you can inspec the directed acyclic graph (DAG).

To get the DAG for a particular process, you can run the following to save an image of the DAG.

```bash
snakemake --snakefile snakefile --use-conda --conda-frontend conda --configfile config.yaml --cores 1 --dag | dot -Tsvg > dag.svg
```

# Activity: Short-Read Processing and Classification for <i>M. tuberculosis</i>

In this short activity, we will download short-read whole-genome sequencing for <i>Mycobacterium tuberculosis</i>, perform quality-control, and perform taxonomic classification.

Sequencing samples can be contaminated by DNA from humans (either the person from whom the sample was collected or anyone working in the laboratory), environmental bacteria, or other samples being tested in the laboratory. They could also be contaminated with other organisms if the patient is infected with multiple pathogens simultaneously.

## Processing Steps

1. Get quality control statistics on read length and quality using `fastqc` and `seqkit`.
2. Trim sequencing adapters and remove reads that are too short using `fastp`.
4. Perform taxonomic classification (actually we won't do this because it is too memory-intensive, so I have provided the output files) using `kraken2`.

After this, we will inspect the taxonomic classifications of reads to see if they make sense with what we expect.

# 1. Notebook Set Up

In [85]:
import os, glob, re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

out_dir = os.getcwd()

# 2. Run the workflow

Run the following commands to generate all the output files (except the kraken classification files, which I have provided already)

```bash
cd snakemake_tutorial_BST281

# activate the environment
conda activate snakemake_tutorial_BST281

# change the number of cores and RAM available for your local machine
snakemake --snakefile snakefile --use-conda --conda-frontend conda --configfile config.yaml --cores 8 --resources mem_mb=8000
```

# 3. Read Quality Control

## Questions

### i. Plot the average quality of reads, stratified by sample and the read file (6 values total). Also plot the qualities scaled to error rates.

### ii. What do you notice about the quality scoress for R1 vs. R2?

# 4. Taxonomic Classification

For taxonomic classification, we will use the Kraken suite of tools for metagenomic classification from Johns Hopkins. 

These tools use exact-matching of read k-mers against a Kraken database of k-mers from existing genomes.

For full read classification, kraken tools are preferred over other methods (like MetaPhlan) because they contain genome-wide k-mers, rather than only k-mers that distinguish between taxa. So they can be used for both sample-level classification and read-level classification.

Kraken2, the software we will be using, is also very fast.

More about it here:

<ul>
    <li><a href="https://www.nature.com/articles/s41596-022-00738-y" target="_blank">Metagenome analysis using the Kraken software suite
</a></li>
    <li><a href="https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1891-0" target="_blank">Improved metagenomic analysis with Kraken 2</a></li>
</ul>

<b>In each kraken report, there are 6 columns:</b>

1. Percentage of Reads: Percentage of total classified reads assigned to this taxon (including sub-taxa).
2. Number of Reads: Number of reads assigned directly to this taxon (not including sub-taxa).
3. Number of Reads (Including Sub-taxa): Number of reads assigned to this taxon plus all its sub-taxa.
4. Taxonomic Rank Code: Single-letter code indicating taxonomic rank (e.g., U = Unclassified, R = Root, D = Domain, P = Phylum, C = Class, O = Order, F = Family, G = Genus, S = Species).
5. NCBI Taxonomic ID: The NCBI Taxonomy ID for this taxon.
6. Taxon Name: The name of the taxon, indented with leading spaces to show the hierarchy.

## Questions

### iii. What percent of reads map to the following groups in each sample:

<ul>
<li><i>Mycobacterium</i> genus</li>
<li><i>Homo sapiens</i></li>
<li>Unclassified</li>
</ul>

### iv. Where do you think the unclassified reads come from?

### v. What is the most common bacterial contaminant (not in the <i>Mycobacterium</i> genus) in each sample?

### vi. In any of the samples, is the most common genus not <i>Mycobacterium</i>?

### vii. Where do you think these bacterial contaminants come from? Are they biological or technical?

There was an interesting <a href="https://pmc.ncbi.nlm.nih.gov/articles/PMC4228153/" target="_blank">paper</a> published some years ago about contaminating DNA found in PCR and DNA library prep reagents. Many of the organisms from which this DNA originates are commensal bacteria or ubiquitous environmental bacteria.

# 5. Write your own rule

Let's say we want to further investigate the unclassified reads to determine if they are something novel not represented in the standard Kraken database or if they are due to poor quality sequencing. The first step would be to gather the unclassified reads and their associated quality scores.

## Questions

### viii. How many reads are in each of the 6 FASTQ files (after the kraken-classification step)?

Verify that the R1 and R2 files of each sample have the same numbers of reads.

### ix. Extract the 1000th read (by count) from each file and print it and its name out.

Do not use any tools other than Python (Biopython) and bash.

### x. Write a rule to extract the unclassified reads for each sample and write them to new FASTQ files 

The `seqtk` <a href="https://github.com/lh3/seqtk" target="_blank">package</a> will be useful because it can extract reads by name. This package has already been installed in the `read_QC` environment.

There should be two FASTQ files, one for the forward reads and one for the reverse. Add the rule to your `rules.smk` file. Update your `snakefile` accordingly and rerun the workflow. 

Notice that the workflow will only run the new rule because the previous rules nor their output files have been changed.